In [1]:
# Vinicius da Silva - 206734
import gurobipy as gp
import numpy as np
import time

# Parâmetros

In [2]:
# inicializações básicas
J = 100
F = np.random.randint(J, 2*J+1)
L = np.random.randint(5, 11) #usamos 11 pq a função randint é do tipo [menor,maior[
M = np.random.randint(5, 11)
P = np.random.randint(5, 11)

print(f"{J=}, {F=}, {L=}, {M=}, {P=}")

J=100, F=173, L=6, M=8, P=9


In [3]:
# Parâmetros baseados nas inicializações

# demanda do cliente j do produto p
D = np.random.randint(10, 21, size=(J,P))
# material m disponivel na fabrica f
R = np.random.randint(800, 1001, size=(M,F))
#capacidade de produção c da máquina l na fábrica f
C = np.random.randint(80, 101, size=(L,F))

# material m necessario para produzir produto p na maquina l
mr = np.random.randint(1, 6, size=(M,P,L))
# custo de produção do produto p usando a máquina l na fábrica f
cp = np.random.randint(10, 101, size=(P,L,F))
# custo de transporte do produto p da fábrica f até o cliente j
ct = np.random.randint(10, 21, size=(P,F,J))

# Variáveis de decisão

In [4]:
# Com base no modelo visto em sala, vamos usar o custo produzido e custo transportado de forma separada.
# Assim também vamos utilizar a qtd produzida e transportada de forma separada, dado que a minimização da função 
# é baseada na soma das duas variáveis

start_time = time.time()
model = gp.Model()
px = model.addMVar((P, L, F), lb=0.0, vtype=gp.GRB.CONTINUOUS, name="qtd_produzida")
py = model.addMVar((P, F, J), lb=0.0, vtype=gp.GRB.CONTINUOUS, name="qtd_transportada")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-29


In [5]:
print(f"Quantidade de Variáveis  = {sum(px.shape) + sum(py.shape)}")

Quantidade de Variáveis  = 470


# Restrições

In [6]:
# Demanda do cliente: A demanda dos clientes j por cada produto p deve ser igual a soma das toneladas 
# de cada produto p transportadas de cada fábrica f
r0 = model.addConstrs((
    gp.quicksum(py[p][f][j] for f in range(F)) == D[j][p]
    for p in range(P)
    for j in range(J)
), name='demanda');

In [7]:
# Quantidade de matéria prima: A quantidade de matéria prima m disponível em cada fábrica f deve ser maior ou igual
# à quantidade necessária para produzir todos os pedidos feitos (quantidade produzida * quantidade necessaria)
r1 = model.addConstrs((
    gp.quicksum(mr[m][p][l] * px[p][l][f] for p in range(P) for l in range(L)) <= R[m][f]
    for m in range(M)
    for f in range(F)
), name='material_disponivel');

In [8]:
# Capacidade disponível de produção: A capacidade disponível de produção deve ser maior ou igual a quantidade total 
# de produção que deve ser realizada
r2 = model.addConstrs((
    gp.quicksum(px[p][l][f] for p in range(P)) <= C[l][f]
    for l in range(L)
    for f in range(F)
), name='capacidade_disponivel');

In [9]:
# Relação entre quantidade transportada e produzida: A quantidade transportada deve ser igual a quantidade produzida
r3 = model.addConstrs((
    gp.quicksum(px[p][l][f] for l in range(L)) == gp.quicksum(py[p][f][j] for j in range(J))
    for p in range(P)
    for f in range(F)
), name='relacao_produzida_transportada');

In [10]:
print(f"Quantidade de Restrições = {len(r0) + len(r1) + len(r2) + len(r3)}")

Quantidade de Restrições = 4879


# Função objetivo

In [11]:
# o objetivo é minimizar o custo total de produção de todos os produtos em todas as máquinas de todas as fábricas
# somado com o custo total de transporte de todos os produtos levados de todas as fábricas até todos os clientes

custo_producao = gp.quicksum(px[p][l][f] * cp[p][l][f] for f in range(F) for l in range(L) for p in range(P))
custo_transporte = gp.quicksum(py[p][f][j] * ct[p][f][j] for j in range(J) for f in range(F) for p in range(P))

model.setObjective(custo_producao + custo_transporte, sense=gp.GRB.MINIMIZE)

# Resultado

In [12]:
model.optimize()
end_time = time.time()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4879 rows, 165042 columns and 404820 nonzeros
Model fingerprint: 0x1043f664
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.24s
Presolved: 4879 rows, 165042 columns, 404820 nonzeros

Ordering time: 0.13s

Barrier statistics:
 AA' NZ     : 1.906e+05
 Factor NZ  : 1.304e+06 (roughly 80 MB of memory)
 Factor Ops : 7.860e+08 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.71873525e+08  0.00000000e+00  1.64e+04 0.00e+00  4.24e+03     1s
   1   2.77426312e+07 -1.87622376e+06  1.71e+03 4.26e-14  4.4

In [13]:
print(f"Número de Iterações      = {int(model.IterCount)}")
print(f"Tempo de Execução        = {end_time - start_time} s")
print(f"Custo da Solução         = {model.objVal}")

Número de Iterações      = 7871
Tempo de Execução        = 52.36984944343567 s
Custo da Solução         = 277843.2
